In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk

## Preprocessing

In [2]:
events = [
    "canada_wildfires_2016", "cyclone_idai_2019", "ecuador_earthquake_2016",
    "hurricane_harvey_2017", "hurricane_irma_2017", "hurricane_maria_2017",
    "hurricane_matthew_2016", "italy_earthquake_aug_2016", "kaikoura_earthquake_2016",
    "puebla_mexico_earthquake_2017", "srilanka_floods_2017"
]

dataframes = {event: {} for event in events}

# Loop through each event and load train, dev, and test sets
for event in events:
    for set_type in ['train', 'dev', 'test']:
        file_path = f'../data/HumAID/{event}/{event}_{set_type}.tsv'
        # Load the dataset and store it in the dictionary under the appropriate event and set type
        dataframes[event][set_type] = pd.read_csv(file_path, sep='\t')

In [3]:
dataframes['canada_wildfires_2016']['test']

,tweet_id,tweet_text,class_label
0,728674116773904384,RT @FoothillsFCU23: In response the to the #Fo...,rescue_volunteering_or_donation_effort
1,729787427829612544,Redcross is offering charitable donation recei...,rescue_volunteering_or_donation_effort
2,730510385544085505,RT @globeandmail: Red Cross to transfer $50-mi...,rescue_volunteering_or_donation_effort
3,733705874594746368,Live: Emergency operations briefing on north A...,other_relevant_information
4,730606066023665665,"$9bn fire damage to Fort McMurray, ‘the beast’...",infrastructure_and_utility_damage
...,...,...,...
440,729062171993374720,I feel sad Mom &amp; I r donating money to the...,rescue_volunteering_or_donation_effort
441,728733841230311425,This is the best way to help. The Red Cross wi...,rescue_volunteering_or_donation_effort
442,730078294259961856,RT @TheGrayGroup: Donations for Fort McMurray ...,rescue_volunteering_or_donation_effort
443,730002964035723264,Local volleyball team members raise money to h...,rescue_volunteering_or_donation_effort


In [4]:
train_dfs = []
dev_dfs = []
test_dfs = []

# Iterate over each event and set type, adding the event name as a column
for event, sets in dataframes.items():
    for set_type, df in sets.items():
        df['event'] = event  # Add the event name as a column
        
        # Append the DataFrame to the appropriate list based on the set type
        if set_type == 'train':
            train_dfs.append(df)
        elif set_type == 'dev':
            dev_dfs.append(df)
        elif set_type == 'test':
            test_dfs.append(df)

# Concatenate the lists of DataFrames into three large DataFrames
train_df = pd.concat(train_dfs, ignore_index=True)
val_df = pd.concat(dev_dfs, ignore_index=True)
test_df = pd.concat(test_dfs, ignore_index=True)

In [5]:
train_df['class_label'].value_counts()

rescue_volunteering_or_donation_effort    8212
other_relevant_information                5143
infrastructure_and_utility_damage         4151
sympathy_and_support                      3253
injured_or_dead_people                    2578
caution_and_advice                        1533
displaced_people_and_evacuations          1496
not_humanitarian                          1284
requests_or_urgent_needs                  1139
missing_or_found_people                     13
Name: class_label, dtype: int64

## Topic Classification

In [6]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

/Users/brandonluong/opt/anaconda3/lib/python3.9/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [21]:
sequence_to_classify = "one day I will see the world"
candidate_labels = ['travel', 'cooking', 'dancing']
classifier(sequence_to_classify, candidate_labels)

{'sequence': 'one day I will see the world',
 'labels': ['travel', 'dancing', 'cooking'],
 'scores': [0.9938650727272034, 0.003273798618465662, 0.0028610429726541042]}